## Function Definitions
Functions in PowerShell are defined with the `function` keyword, and should follow PascalCase (where each word is capitalized) and start with an approved verb:

In [ ]:
Get-Verb | Sort-Object -Property Verb

In [ ]:
# example function definition
function Get-HomeDirs {
    Get-ChildItem $HOME -Attribute d+!h | 
        ForEach-Object { if($_.BaseName -notmatch "^\.") { $_ } }
}
Get-HomeDirs

## Function Parameters
Function parameters are defined inside of a `param` statement using the syntax of `[type]$paramterName`, like so:

In [ ]:
function Get-AccountInfo {
    param(
        [String]$user
    )
    # queries all user and group accounts on the system
    Get-CimInstance -ClassName Win32_Account | Where-Object { $_.SIDType -eq 1 -And $_.Name -like "*$user*" } | Select-Object Caption, SID
}
Get-AccountInfo -user "admin"

As can be seen above, the parameter is added as a named parameter to the function and made available via the `-user` switch.

Function parameters can be 'splatted'; combined into named splat variable and fed into a function all at once using the `@` symbol in place of the `$` when using the variable.

In [ ]:
# the splat variable here is defined using a hashtable - note that the hashtable starts with @ symbol, but is distinct from a splat variable
$dirSearch = @{
    Path=$HOME
    Attributes="!Hidden+Directory"
    Exclude=".*"
}

Get-ChildItem @dirSearch


Function parameters can be decorated with _attributes_ to change how they are processed. An _attribute_ is similar to a type, but rather than describing an object like a type does, an attribute describes _metadata_ that can be attached to a function or piece of code and provide instruction on how that code takes processes the data it is passed.  

In [ ]:
function SayHello {
    param(
    # makes the following parameter mandatory and provide a help message
    [Parameter(Mandatory,HelpMessage="Please provide a name!")]
    [string]$name
    )
    Write-Host  "Hi $name"
}

# enter !? in the message box that appears at the top of vscode to show the help message
SayHello 

## Making Functions Work with the Pipeline

The PowerShell pipeline does not execute functions sequentially, but in fact executes them simultanously; each function then contains code which processes the input the function receives via the pipeline.  
This code is contained within specially named blocks within each function. These include:
- The `begin` block, which executes _once_ before the pipeline starts to process any data. Code within the `begin` block can be used to initialize things within the function.
- The `process` block, which executes repeatedly, almost like the ForEach cmdlet, once per incoming pipeline object.
- Finally the `end` block, which executes once after all pipeline elements have been processed. The `end` block can be used to clean things up, like temporary files.  
<sup>_source: https://powershell.one/powershell-internals/scriptblocks/powershell-pipeline_</sup>

As mentioned above, the `process` block is where any input to the function from the pipeline ends up, and the usual `$_` character can be used to represent the input in the function body.

In [ ]:
function demo {
    begin {
        "BEGIN: runs once at the start"
    }
    process {
        "PROCESS: " + $_
    }
    end {
        "END: runs once at the end"
    }
}

1,2,3,4 | demo

## Advanced Functions

As mentioned previously there are two varieties of cmdlet in PowerShell: binary cmdlets which are compiled .NET code, and script cmdlets which use PowerShell scripts and functions. Script cmdlets are also called _advanced functions_.

To turn a function into an advanced function, i.e a function that can be used as a cmdlet, the function needs to be written with `[CmdletBinding()]` attribute. This attribute provides the advanced function cmdlet with access to PowerShells common parameter set; these are parameters that are available to any cmdlet and do not have to be defined when writing the advanced function cmdlet.

In [29]:
function MakeAdvanced {
    [CmdletBinding()]
    Param()
    Write-Debug "Debug Output"
    Write-Verbose "Verbose Output"
}

# Write-Output "As you can see in the help text below, under the SYNTAX heading, [CommonParameters] are now accepted by this function"
Get-Help MakeAdvanced

# The full list of common parameters is out of scope of this notebook, but an example parameter is the -Debug:$true parameter or simply -Debug
MakeAdvanced -Debug

# Or the -Verbose parameter
MakeAdvanced -Verbose


NAME
    MakeAdvanced
    
SYNTAX
    MakeAdvanced [<CommonParameters>]
    

ALIASES
    None
    

REMARKS
    None

DEBUG: Debug Output
VERBOSE: Verbose Output



Advanced functions can take pipeline input by decorating a parameter with the [Parameter()] attribute and using either `ValueFromPipeline` (which performs parameter binding by value) or `ValueFromPipeLineByPropertyName` (which performs parameter binding by property).

In [21]:
function GetHalf {
    Param (
    # this attributes binds the below parameter to a property of the input object with the same name
    [Parameter(ValueFromPipelineByPropertyName)]
    [int] $number
    )
    $result = ($number / 2)
    Write-Output "half of $number is $result"
}

[PSCustomObject]@{ number = 22 }  | GetHalf

# both [CmdletBinding()] and [Parameter()] attributes register the function as an advanced function, as can be seen in the help for this function
Get-Help GetHalf

half of 22 is 11

NAME
    GetHalf
    
SYNTAX
    GetHalf [[-number] <int>] [<CommonParameters>]
    

ALIASES
    None
    

REMARKS
    None


